In [179]:
from scipy.stats import skew
#from IPython.display import display
#import matplotlib.pyplot as plt
#import seaborn as sns

import glob, re
import numpy as np
import pandas as pd
from sklearn import *
#from sklearn.model_selection import cross_val_score, train_test_split
#from sklearn.preprocessing import StandardScaler
#from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
#from sklearn.metrics import mean_squared_error, make_scorer
#from sklearn.model_selection import StratifiedKFold, ParameterGrid
import datetime as dt
from datetime import datetime
from xgboost import XGBRegressor
import xgboost as xgb

from keras.layers import Embedding, Input, Dense
import keras
import keras.backend as K

import matplotlib.pyplot as plt
from keras.models import Sequential, Model

In [180]:
data = {
'tra' : pd.read_csv('/Users/suzukishinji/kaggle/recluit/tra_gw.csv'),#1回目は513店舗#2回目は316
'tes' : pd.read_csv('/Users/suzukishinji/kaggle/recluit/tes_gw.csv'),
    
'ar':pd.read_csv('/Users/suzukishinji/kaggle/recluit/air_reserve.csv'),
'as' : pd.read_csv('/Users/suzukishinji/kaggle/recluit/air_store_info.csv'),
'hol' : pd.read_csv('/Users/suzukishinji/kaggle/recluit/date_info.csv').rename(columns ={'calendar_date':'visit_date'}),
'hr' : pd.read_csv('/Users/suzukishinji/kaggle/recluit/hpg_reserve.csv'),
'hs' : pd.read_csv('/Users/suzukishinji/kaggle/recluit/hpg_store_info.csv'),
'id' : pd.read_csv('/Users/suzukishinji/kaggle/recluit/store_id_relation.csv')}

In [181]:
print(data['tra'].shape)
print(data['tes'].shape)

(6793, 3)
(18062, 2)


In [182]:
data['hr'] = pd.merge(data['hr'], data['id'], how = 'inner', on = ['hpg_store_id'])

In [183]:
for df in ['ar', 'hr']:
    data[df]['visit_datetime'] = pd.to_datetime(data[df]['visit_datetime'])
    data[df]['visit_dow'] = data[df]['visit_datetime'].dt.dayofweek
    data[df]['visit_datetime'] = data[df]['visit_datetime'].dt.date
    data[df]['reserve_datetime'] = pd.to_datetime(data[df]['reserve_datetime'])
    data[df]['reserve_datetime'] = data[df]['reserve_datetime'].dt.date
    data[df]['reserve_datetime_diff'] = data[df].apply(lambda r:(r['visit_datetime']- r['reserve_datetime']).days, axis = 1)
    data[df] = data[df][data[df]['reserve_datetime_diff'] > data[df]['visit_dow']]
    tmp1 = data[df].groupby(['air_store_id', 'visit_datetime'], as_index =False)[['reserve_datetime_diff', 'reserve_visitors']].sum().rename(columns = {'visit_datetime':'visit_date', 'reserve_datetime_diff':'rs1', 'reserve_visitors':'rv1'})
    tmp2 = data[df].groupby(['air_store_id', 'visit_datetime'], as_index =False)[['reserve_datetime_diff', 'reserve_visitors']].mean().rename(columns = {'visit_datetime':'visit_date', 'reserve_datetime_diff':'rs2', 'reserve_visitors':'rv2'})
    data[df] = pd.merge(tmp1, tmp2, how = 'inner', on = ['air_store_id', 'visit_date'])

In [184]:
data['tra']['visit_date'] = pd.to_datetime(data['tra']['visit_date'])
data['tra']['dow'] = data['tra']['visit_date'].dt.dayofweek
data['tra']['year'] = data['tra']['visit_date'].dt.year
data['tra']['month'] = data['tra']['visit_date'].dt.month
data['tra']['visit_date'] = data['tra']['visit_date'].dt.date

In [185]:
data['tes'].head()

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,0
1,air_00a91d42b08b08d9_2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,0
3,air_00a91d42b08b08d9_2017-04-26,0
4,air_00a91d42b08b08d9_2017-04-27,0


In [186]:
data['tes']['visit_date'] = data['tes']['id'].map(lambda x: str(x).split('_')[2])
data['tes']['air_store_id'] = data['tes']['id'].map(lambda x: '_'.join(x.split('_')[:2]))
data['tes']['visit_date'] = pd.to_datetime(data['tes']['visit_date'])
data['tes']['dow'] = data['tes']['visit_date'].dt.dayofweek
data['tes']['year'] = data['tes']['visit_date'].dt.year
data['tes']['month'] = data['tes']['visit_date'].dt.month
data['tes']['visit_date'] = data['tes']['visit_date'].dt.date

In [187]:
unique_stores = data['tes']['air_store_id'].unique()

In [188]:
stores = pd.concat([pd.DataFrame({'air_store_id':unique_stores, 'dow':[i]*len(unique_stores)}) for i in range(7)], axis =0, ignore_index = True).reset_index(drop = True)

In [189]:
#曜日だけでなく、月も追加
stores_m = pd.concat([pd.DataFrame({'air_store_id':unique_stores, 'month':[i]*len(unique_stores)}) for i in range(1,13)], axis =0, ignore_index = True).reset_index(drop = True)
stores = pd.merge(stores_m, stores,on=('air_store_id'), how='left')

In [190]:
tmp = data['tra'].groupby(['air_store_id', 'dow'], as_index = False)['visitors'].min().rename(columns = {'visitors':'min_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'dow'])
tmp = data['tra'].groupby(['air_store_id', 'dow'], as_index = False)['visitors'].mean().rename(columns = {'visitors':'mean_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'dow'])
tmp = data['tra'].groupby(['air_store_id', 'dow'], as_index = False)['visitors'].median().rename(columns = {'visitors':'median_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'dow'])
tmp = data['tra'].groupby(['air_store_id', 'dow'], as_index = False)['visitors'].max().rename(columns = {'visitors':'max_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'dow'])
tmp = data['tra'].groupby(['air_store_id', 'dow'], as_index = False)['visitors'].count().rename(columns = {'visitors':'count_observations'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'dow'])

In [191]:
#曜日だけでなく、ID×月も追加
tmp = data['tra'].groupby(['air_store_id', 'month'], as_index = False)['visitors'].min().rename(columns = {'visitors':'m_min_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'month'])
tmp = data['tra'].groupby(['air_store_id', 'month'], as_index = False)['visitors'].mean().rename(columns = {'visitors':'m_mean_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'month'])
tmp = data['tra'].groupby(['air_store_id', 'month'], as_index = False)['visitors'].median().rename(columns = {'visitors':'m_median_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'month'])
tmp = data['tra'].groupby(['air_store_id', 'month'], as_index = False)['visitors'].max().rename(columns = {'visitors':'m_max_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'month'])
tmp = data['tra'].groupby(['air_store_id', 'month'], as_index = False)['visitors'].count().rename(columns = {'visitors':'m_count_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'month'])

In [192]:
tmp = data['as'].groupby(['air_genre_name','air_area_name'], as_index = False)['air_store_id'].count().rename(columns = {'air_store_id':'genre_in_area'})
data['as']= pd.merge(data['as'], tmp, how ='left', on = ['air_genre_name','air_area_name'])

In [193]:
stores = pd.merge(stores, data['as'], how= "left", on = ['air_store_id'])

In [194]:
stores['air_genre_name'] = stores['air_genre_name'].map(lambda x: str(str(x).replace('/', ' ')))
stores['air_area_name'] = stores['air_area_name'].map(lambda x: str(str(x).replace('-', ' ')))

In [195]:
lbl = preprocessing.LabelEncoder()
for i in range(10):
    stores['air_genre_name' + str(i)] = lbl.fit_transform(stores['air_genre_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ' '))
    stores['air_area_name' + str(i)] = lbl.fit_transform(stores['air_area_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ' '))
stores['air_genre_name'] = lbl.fit_transform(stores['air_genre_name'])
stores['air_area_name'] = lbl.fit_transform(stores['air_area_name'])

In [196]:
#土日フラグ(day_of_week_1)と、休日前(holi_2)フラグを追加
data['hol']['day_of_week_1']= data['hol']['day_of_week'].replace(['Saturday', 'Sunday','Monday','Tuesday','Wednesday','Thursday','Friday'],['1', '1','0','0','0','0','0']).astype('int')
data['hol']['holi_2'] = data['hol'][['holiday_flg', 'day_of_week_1']].sum(axis = 1)
data['hol']['holi_2'] = data['hol']['holi_2'].apply( lambda x: 0 if x < 1 else 1 )
data['hol']['holi_2'] = data['hol']['holi_2'].shift(-1)
data['hol']['holi_2'] = data['hol']['holi_2'].fillna(1)
data['hol']['holi_2'] = data['hol']['holi_2'].astype('int')

In [197]:
data['hol'].head()

,visit_date,day_of_week,holiday_flg,day_of_week_1,holi_2
0,2016-01-01,Friday,1,0,1
1,2016-01-02,Saturday,1,1,1
2,2016-01-03,Sunday,1,1,0
3,2016-01-04,Monday,0,0,0
4,2016-01-05,Tuesday,0,0,0


In [198]:
data['hol']['visit_date'] = pd.to_datetime(data['hol']['visit_date'])
data['hol']['day_of_week'] = lbl.fit_transform(data['hol']['day_of_week'])
data['hol']['visit_date'] = data['hol']['visit_date'].dt.date
train = pd.merge(data['tra'], data['hol'], how ='left', on = ['visit_date'])
test = pd.merge(data['tes'], data['hol'], how ='left', on = ['visit_date'])

In [199]:
#曜日と月でmerge
#train = pd.merge(train, stores, how ='left', on = ['air_store_id', 'dow','month'])
train = pd.merge(train, stores, how ='inner', on = ['air_store_id', 'dow','month'])
test = pd.merge(test, stores, how ='left', on = ['air_store_id', 'dow','month'])

In [200]:
#ID×休日前でのvisitorsの平均、中央値等を追加
tmp = train.groupby(['air_store_id','holi_2'], as_index = False)['visitors'].min().rename(columns = {'visitors':'h_min_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','holi_2'])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','holi_2'])
tmp = train.groupby(['air_store_id','holi_2'], as_index = False)['visitors'].mean().rename(columns = {'visitors':'h_mean_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','holi_2'])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','holi_2'])
tmp = train.groupby(['air_store_id','holi_2'], as_index = False)['visitors'].median().rename(columns = {'visitors':'h_median_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','holi_2'])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','holi_2'])
tmp = train.groupby(['air_store_id','holi_2'], as_index = False)['visitors'].max().rename(columns = {'visitors':'h_max_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','holi_2'])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','holi_2'])
tmp = train.groupby(['air_store_id','holi_2'], as_index = False)['visitors'].count().rename(columns = {'visitors':'h_count_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','holi_2'])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','holi_2'])

In [201]:
train.tail()

,air_store_id,visit_date,visitors,dow,year,month,day_of_week,holiday_flg,day_of_week_1,holi_2,...,air_area_name7,air_genre_name8,air_area_name8,air_genre_name9,air_area_name9,h_min_visitors,h_mean_visitors,h_median_visitors,h_max_visitors,h_count_visitors
6735,air_c7d30ab0e07f31d5,2016-05-06,8,4,2016,5,0,0,0,1,...,0,0,0,0,0,8,15.083333,14.5,24,12
6736,air_c7d30ab0e07f31d5,2016-05-13,20,4,2016,5,0,0,0,1,...,0,0,0,0,0,8,15.083333,14.5,24,12
6737,air_c7d30ab0e07f31d5,2016-05-07,11,5,2016,5,2,0,1,1,...,0,0,0,0,0,8,15.083333,14.5,24,12
6738,air_c7d30ab0e07f31d5,2016-05-14,10,5,2016,5,2,0,1,1,...,0,0,0,0,0,8,15.083333,14.5,24,12
6739,air_24e8414b9b07decb,2017-04-22,5,5,2017,4,2,0,1,1,...,0,0,0,0,0,5,5.000000,5.0,5,1


In [202]:
#combine = [train, test]
#gw_list = ['2017-04-22','2017-04-21','2017-04-20','2017-04-19','2017-04-18','2017-04-17','2017-04-16','2017-04-15','2017-04-14','2017-04-13','2017-04-12','2017-04-11','2017-04-10','2017-04-09','2017-04-08']
#post_gw_list=[]
#train['gw_flg'] = 0
#train['post_gw_flg'] = 0
#test['gw_flg'] = 0
#test['post_gw_flg'] = 0
#update_gw_list = [["0" for i in range(3)] for j in range(len(gw_list))]
#update_post_gw_list = [["0" for i in range(3)] for j in range(len(post_gw_list))]

In [203]:
#from datetime import date
#for index, gw_date in enumerate(gw_list):
#    temp_list = gw_date.split("-")
#    for col_i, temp_figure in enumerate(temp_list):
#        update_gw_list[index][col_i]=int(temp_figure)
#        
#    #print("{}  {}  {}".format(update_list[index][0],update_list[index][1],update_list[index][2]))
#    #
#for index, gw_date in enumerate(post_gw_list):
#    temp_list = gw_date.split("-")
#    for col_i, temp_figure in enumerate(temp_list):
#        update_post_gw_list[index][col_i]=int(temp_figure)
#
#for dataset in combine:
#    for index in range(len(update_gw_list)):
#        dataset.loc[dataset.visit_date == date(update_gw_list[index][0],update_gw_list[index][1],update_gw_list[index][2]), 'gw_flg'] = 1
#        
#for dataset in combine:
#    for index in range(len(update_post_gw_list)):
#        dataset.loc[dataset.visit_date == date(update_post_gw_list[index][0],update_post_gw_list[index][1],update_post_gw_list[index][2]), 'post_gw_flg'] = 1

In [204]:
#tmp = train.groupby(['air_store_id','gw_flg'], as_index = False)['visitors'].min().rename(columns = {'visitors':'gw_min_visitors'})
#train = pd.merge(train, tmp, how ='left', on = ['air_store_id','gw_flg'])
#test = pd.merge(test, tmp, how ='left', on = ['air_store_id','gw_flg'])
#tmp = train.groupby(['air_store_id','gw_flg'], as_index = False)['visitors'].mean().rename(columns = {'visitors':'gw_mean_visitors'})
#train = pd.merge(train, tmp, how ='left', on = ['air_store_id','gw_flg'])
#test = pd.merge(test, tmp, how ='left', on = ['air_store_id','gw_flg'])
#tmp = train.groupby(['air_store_id','gw_flg'], as_index = False)['visitors'].median().rename(columns = {'visitors':'gw_median_visitors'})
#train = pd.merge(train, tmp, how ='left', on = ['air_store_id','gw_flg'])
#test = pd.merge(test, tmp, how ='left', on = ['air_store_id','gw_flg'])
#tmp = train.groupby(['air_store_id','gw_flg'], as_index = False)['visitors'].max().rename(columns = {'visitors':'gw_max_visitors'})
#train = pd.merge(train, tmp, how ='left', on = ['air_store_id','gw_flg'])
#test = pd.merge(test, tmp, how ='left', on = ['air_store_id','gw_flg'])
#tmp = train.groupby(['air_store_id','gw_flg'], as_index = False)['visitors'].count().rename(columns = {'visitors':'gw_count_visitors'})
#train = pd.merge(train, tmp, how ='left', on = ['air_store_id','gw_flg'])
#test = pd.merge(test, tmp, how ='left', on = ['air_store_id','gw_flg'])

In [205]:
#tmp = train.groupby(['air_store_id','post_gw_flg'], as_index = False)['visitors'].min().rename(columns = {'visitors':'pgw_min_visitors'})
#train = pd.merge(train, tmp, how ='left', on = ['air_store_id','post_gw_flg'])
#test = pd.merge(test, tmp, how ='left', on = ['air_store_id','post_gw_flg'])
#tmp = train.groupby(['air_store_id','post_gw_flg'], as_index = False)['visitors'].mean().rename(columns = {'visitors':'pgw_mean_visitors'})
#train = pd.merge(train, tmp, how ='left', on = ['air_store_id','post_gw_flg'])
#test = pd.merge(test, tmp, how ='left', on = ['air_store_id','post_gw_flg'])
#tmp = train.groupby(['air_store_id','post_gw_flg'], as_index = False)['visitors'].median().rename(columns = {'visitors':'pgw_median_visitors'})
#train = pd.merge(train, tmp, how ='left', on = ['air_store_id','post_gw_flg'])
#test = pd.merge(test, tmp, how ='left', on = ['air_store_id','post_gw_flg'])
#tmp = train.groupby(['air_store_id','post_gw_flg'], as_index = False)['visitors'].max().rename(columns = {'visitors':'pgw_max_visitors'})
#train = pd.merge(train, tmp, how ='left', on = ['air_store_id','post_gw_flg'])
#test = pd.merge(test, tmp, how ='left', on = ['air_store_id','post_gw_flg'])
#tmp = train.groupby(['air_store_id','post_gw_flg'], as_index = False)['visitors'].count().rename(columns = {'visitors':'pgw_count_visitors'})
#train = pd.merge(train, tmp, how ='left', on = ['air_store_id','post_gw_flg'])
#test = pd.merge(test, tmp, how ='left', on = ['air_store_id','post_gw_flg'])

In [206]:
for df in ['ar','hr']:
    train = pd.merge(train, data[df], how='left', on=['air_store_id','visit_date']) 
    test = pd.merge(test, data[df], how='left', on=['air_store_id','visit_date'])

In [207]:
train['id'] = train.apply(lambda r: '_'.join([str(r['air_store_id']), str(r['visit_date'])]), axis=1)

In [208]:
train['total_reserv_sum'] = train['rv1_x'] + train['rv1_y']
train['total_reserv_mean'] = (train['rv2_x'] + train['rv2_y'])/2
train['total_reserv_dt_diff_mean'] = (train['rs2_x'] + train['rs2_y'])/2

In [209]:
test['total_reserv_sum'] = test['rv1_x'] + test['rv1_y']
test['total_reserv_mean'] = (test['rv2_x'] + test['rv2_y'])/2
test['total_reserv_dt_diff_mean'] = (test['rs2_x'] + test['rs2_y'])/2

In [210]:
train['date_int'] = train['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
test['date_int'] = test['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
train['var_max_lat'] = train['latitude'].max() - train['latitude']
train['var_max_long'] = train['longitude'].max() - train['longitude']
test['var_max_lat'] = test['latitude'].max() - test['latitude']
test['var_max_long'] = test['longitude'].max() - test['longitude']
train['lon_plus_lat'] = train['longitude'] + train['latitude'] 
test['lon_plus_lat'] = test['longitude'] + test['latitude']

In [211]:

lbl = preprocessing.LabelEncoder()
train['air_store_id2'] = lbl.fit_transform(train['air_store_id'])
test['air_store_id2']= lbl.fit_transform(test['air_store_id'])

In [212]:
train['lon_plus_lat'] = train['longitude'] + train['latitude'] 
test['lon_plus_lat'] = test['longitude'] + test['latitude']

In [213]:
col = [c for c in train if c not in ['id', 'air_store_id', 'visit_date', 'visitors']]
train = train.fillna(-1)
test = test.fillna(-1)

In [214]:
ntrain = train.shape[0]
ntest = test.shape[0]

In [215]:
all_data = pd.concat([train, test])

In [216]:
#指数移動平均の追加。これはなくても良いかも
#https://www.kaggle.com/c/recruit-restaurant-visitor-forecasting/discussion/46179#266344
#def calc_shifted_ewm(series, alpha, adjust=True):
#    return series.shift().ewm(alpha=alpha, adjust=adjust).mean()

#train['ewm'] = train.groupby(['air_store_id', 'dow']).apply(lambda g: calc_shifted_ewm(g['visitors'], 0.1)).sort_index(level=['air_store_id']).values

In [217]:
#以下、気象データの追加
df_air_store_weather_station = pd.read_csv('./air_store_info_with_nearest_active_station.csv')

In [218]:
cols = ['air_store_id', 'station_id', 'station_latitude', 'station_longitude', 'station_vincenty', 'station_great_circle']
all_data = pd.merge(all_data, df_air_store_weather_station[cols], on='air_store_id', how='left')

In [219]:
combine = all_data
filenames = []
df_weather = None
for station_id in combine['station_id'].unique():
    fn = f"/Users/suzukishinji/kaggle/recluit/1-1-16_5-31-17_Weather/{station_id}.csv"
    if not fn in filenames:
        df = pd.read_csv(fn)
        df['station_id'] = station_id
        if df_weather is None:
            df_weather = df
        else:
            df_weather = pd.concat([df_weather, df])
        del df

        filenames.append(fn)
    else:
        continue

In [220]:
#欠損値を平均で穴埋め（median, ffillで試すも特に差は出なかった）
df_weather = df_weather.fillna(df_weather.mean())

In [221]:
df_weather = df_weather.rename(columns={'calendar_date': 'visit_date'})

In [222]:
df_weather['visit_date'] = pd.to_datetime(df_weather['visit_date'])
df_weather['visit_date'] = df_weather['visit_date'].dt.date

In [223]:
#なんとなく対数化
df_weather['precipitation'] = np.log1p(df_weather['precipitation'])

In [224]:
#使いそうなデータだけ結合（その他の気象データは試していません。特に意味はなし）
cols = ['station_id', 
    'visit_date', 
    'precipitation', 
    'hours_sunlight',
    'avg_temperature',
    'high_temperature',
    'low_temperature']

combine = pd.merge(combine, df_weather[cols], on=['station_id', 'visit_date'], how='left')

In [225]:
#降水量をカテゴリ化
def simplify_pre(df):
    df.precipitation = df.precipitation.fillna(0)
    bins = ( -1, 0.01, 2,  5)
    group_names = ['1', '2', '3']
    categories = pd.cut(df.precipitation, bins, labels=group_names)
    df.precipitation = categories
    return df

In [226]:
combine = simplify_pre(combine)

In [227]:
all_data = combine

In [228]:
#不要そうなデータを削除
drop_col =['station_id', 'station_latitude','station_longitude','station_vincenty', 'station_great_circle','hours_sunlight','high_temperature','low_temperature']
all_data = all_data.drop(drop_col, axis = 1)

In [229]:
train = all_data[:ntrain]
test = all_data[ntrain:]

In [230]:
train['date_int'] = train['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
test['date_int'] = test['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
train['var_max_lat'] = train['latitude'].max() - train['latitude']
train['var_max_long'] = train['longitude'].max() - train['longitude']
test['var_max_lat'] = test['latitude'].max() - test['latitude']
test['var_max_long'] = test['longitude'].max() - test['longitude']

train['lon_plus_lat'] = train['longitude'] + train['latitude'] 
test['lon_plus_lat'] = test['longitude'] + test['latitude']

/Users/suzukishinji/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/suzukishinji/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/suzukishinji/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

In [231]:
#ID×降水量で平均、中央値等を追加
tmp = train.groupby(['air_store_id','precipitation'], as_index = False)['visitors'].min().rename(columns = {'visitors':'p_min_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','precipitation'])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','precipitation'])

tmp = train.groupby(['air_store_id','precipitation'], as_index = False)['visitors'].mean().rename(columns = {'visitors':'p_mean_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','precipitation'])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','precipitation'])

tmp = train.groupby(['air_store_id','precipitation'], as_index = False)['visitors'].median().rename(columns = {'visitors':'p_median_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','precipitation',])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','precipitation'])

tmp = train.groupby(['air_store_id','precipitation'], as_index = False)['visitors'].max().rename(columns = {'visitors':'p_max_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','precipitation'])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','precipitation'])

tmp = train.groupby(['air_store_id','precipitation'], as_index = False)['visitors'].count().rename(columns = {'visitors':'p_count_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','precipitation'])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','precipitation'])

In [232]:
#countをLabel Encoder化。なんとなく試してみたら結果が良かった。
lbl = preprocessing.LabelEncoder()
train['count_visitors'] = lbl.fit_transform(train['count_observations']) 
test['count_visitors']= lbl.fit_transform(test['count_observations'])
train['m_count_visitors'] = lbl.fit_transform(train['m_count_visitors'])
test['m_count_visitors']= lbl.fit_transform(test['m_count_visitors'])
train['h_count_reserves'] = lbl.fit_transform(train['h_count_visitors'])
test['h_count_reserves']= lbl.fit_transform(test['h_count_visitors'])
train['p_count_visitors'] = lbl.fit_transform(train['p_count_visitors'])
test['p_count_visitors']= lbl.fit_transform(test['p_count_visitors'])

In [233]:
def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred)**0.5

In [234]:
value_col = ['holiday_flg','min_visitors','mean_visitors','median_visitors','max_visitors','count_observations',
'rs1_x','rv1_x','rs2_x','rv2_x','rs1_y','rv1_y','rs2_y','rv2_y','total_reserv_sum','total_reserv_mean',
'total_reserv_dt_diff_mean','date_int','var_max_lat','var_max_long','lon_plus_lat']

nn_col = value_col + ['dow', 'year', 'month', 'air_store_id2', 'air_area_name', 'air_genre_name',
'air_area_name0', 'air_area_name1', 'air_area_name2', 'air_area_name3', 'air_area_name4',
'air_area_name5', 'air_area_name6', 'air_genre_name0', 'air_genre_name1',
'air_genre_name2', 'air_genre_name3', 'air_genre_name4']

In [235]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18062 entries, 0 to 18061
Data columns (total 76 columns):
air_area_name                18062 non-null int64
air_area_name0               18062 non-null int64
air_area_name1               18062 non-null int64
air_area_name2               18062 non-null int64
air_area_name3               18062 non-null int64
air_area_name4               18062 non-null int64
air_area_name5               18062 non-null int64
air_area_name6               18062 non-null int64
air_area_name7               18062 non-null int64
air_area_name8               18062 non-null int64
air_area_name9               18062 non-null int64
air_genre_name               18062 non-null int64
air_genre_name0              18062 non-null int64
air_genre_name1              18062 non-null int64
air_genre_name2              18062 non-null int64
air_genre_name3              18062 non-null int64
air_genre_name4              18062 non-null int64
air_genre_name5              18062 non-nu

In [236]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6740 entries, 0 to 6739
Data columns (total 76 columns):
air_area_name                6740 non-null int64
air_area_name0               6740 non-null int64
air_area_name1               6740 non-null int64
air_area_name2               6740 non-null int64
air_area_name3               6740 non-null int64
air_area_name4               6740 non-null int64
air_area_name5               6740 non-null int64
air_area_name6               6740 non-null int64
air_area_name7               6740 non-null int64
air_area_name8               6740 non-null int64
air_area_name9               6740 non-null int64
air_genre_name               6740 non-null int64
air_genre_name0              6740 non-null int64
air_genre_name1              6740 non-null int64
air_genre_name2              6740 non-null int64
air_genre_name3              6740 non-null int64
air_genre_name4              6740 non-null int64
air_genre_name5              6740 non-null int64
air_genre_n

In [237]:
X = train.copy()
X_test = test[nn_col].copy()

value_scaler = preprocessing.MinMaxScaler()
for vcol in value_col:
    X[vcol] = value_scaler.fit_transform(X[vcol].values.astype(np.float64).reshape(-1, 1))
    X_test[vcol] = value_scaler.transform(X_test[vcol].values.astype(np.float64).reshape(-1, 1))

X_train = list(X[nn_col].T.as_matrix())
Y_train = np.log1p(X['visitors']).values
nn_train = [X_train, Y_train]
nn_test = [list(X_test[nn_col].T.as_matrix())]
print("Train and test data prepared")

Train and test data prepared


In [238]:
def get_nn_complete_model(train, hidden1_neurons=35, hidden2_neurons=15):
    K.clear_session()

    air_store_id = Input(shape=(1,), dtype='int32', name='air_store_id')
    air_store_id_emb = Embedding(len(train['air_store_id2'].unique()) + 1, 15, input_shape=(1,),
                                 name='air_store_id_emb')(air_store_id)
    air_store_id_emb = keras.layers.Flatten(name='air_store_id_emb_flatten')(air_store_id_emb)

    dow = Input(shape=(1,), dtype='int32', name='dow')
    dow_emb = Embedding(8, 3, input_shape=(1,), name='dow_emb')(dow)
    dow_emb = keras.layers.Flatten(name='dow_emb_flatten')(dow_emb)

    month = Input(shape=(1,), dtype='int32', name='month')
    month_emb = Embedding(13, 3, input_shape=(1,), name='month_emb')(month)
    month_emb = keras.layers.Flatten(name='month_emb_flatten')(month_emb)

    air_area_name, air_genre_name = [], []
    air_area_name_emb, air_genre_name_emb = [], []
    for i in range(7):
        area_name_col = 'air_area_name' + str(i)
        air_area_name.append(Input(shape=(1,), dtype='int32', name=area_name_col))
        tmp = Embedding(len(train[area_name_col].unique()), 3, input_shape=(1,),
                        name=area_name_col + '_emb')(air_area_name[-1])
        tmp = keras.layers.Flatten(name=area_name_col + '_emb_flatten')(tmp)
        air_area_name_emb.append(tmp)

        if i > 4:
            continue
        area_genre_col = 'air_genre_name' + str(i)
        air_genre_name.append(Input(shape=(1,), dtype='int32', name=area_genre_col))
        tmp = Embedding(len(train[area_genre_col].unique()), 3, input_shape=(1,),
                        name=area_genre_col + '_emb')(air_genre_name[-1])
        tmp = keras.layers.Flatten(name=area_genre_col + '_emb_flatten')(tmp)
        air_genre_name_emb.append(tmp)

    air_genre_name_emb = keras.layers.concatenate(air_genre_name_emb)
    air_genre_name_emb = Dense(4, activation='sigmoid', name='final_air_genre_emb')(air_genre_name_emb)

    air_area_name_emb = keras.layers.concatenate(air_area_name_emb)
    air_area_name_emb = Dense(4, activation='sigmoid', name='final_air_area_emb')(air_area_name_emb)
    
    air_area_code = Input(shape=(1,), dtype='int32', name='air_area_code')
    air_area_code_emb = Embedding(len(train['air_area_name'].unique()), 8, input_shape=(1,), name='air_area_code_emb')(air_area_code)
    air_area_code_emb = keras.layers.Flatten(name='air_area_code_emb_flatten')(air_area_code_emb)
    
    air_genre_code = Input(shape=(1,), dtype='int32', name='air_genre_code')
    air_genre_code_emb = Embedding(len(train['air_genre_name'].unique()), 5, input_shape=(1,),
                                   name='air_genre_code_emb')(air_genre_code)
    air_genre_code_emb = keras.layers.Flatten(name='air_genre_code_emb_flatten')(air_genre_code_emb)

    
    holiday_flg = Input(shape=(1,), dtype='float32', name='holiday_flg')
    year = Input(shape=(1,), dtype='float32', name='year')
    min_visitors = Input(shape=(1,), dtype='float32', name='min_visitors')
    mean_visitors = Input(shape=(1,), dtype='float32', name='mean_visitors')
    median_visitors = Input(shape=(1,), dtype='float32', name='median_visitors')
    max_visitors = Input(shape=(1,), dtype='float32', name='max_visitors')
    count_observations = Input(shape=(1,), dtype='float32', name='count_observations')
    rs1_x = Input(shape=(1,), dtype='float32', name='rs1_x')
    rv1_x = Input(shape=(1,), dtype='float32', name='rv1_x')
    rs2_x = Input(shape=(1,), dtype='float32', name='rs2_x')
    rv2_x = Input(shape=(1,), dtype='float32', name='rv2_x')
    rs1_y = Input(shape=(1,), dtype='float32', name='rs1_y')
    rv1_y = Input(shape=(1,), dtype='float32', name='rv1_y')
    rs2_y = Input(shape=(1,), dtype='float32', name='rs2_y')
    rv2_y = Input(shape=(1,), dtype='float32', name='rv2_y')
    total_reserv_sum = Input(shape=(1,), dtype='float32', name='total_reserv_sum')
    total_reserv_mean = Input(shape=(1,), dtype='float32', name='total_reserv_mean')
    total_reserv_dt_diff_mean = Input(shape=(1,), dtype='float32', name='total_reserv_dt_diff_mean')
    date_int = Input(shape=(1,), dtype='float32', name='date_int')
    var_max_lat = Input(shape=(1,), dtype='float32', name='var_max_lat')
    var_max_long = Input(shape=(1,), dtype='float32', name='var_max_long')
    lon_plus_lat = Input(shape=(1,), dtype='float32', name='lon_plus_lat')

    date_emb = keras.layers.concatenate([dow_emb, month_emb, year, holiday_flg])
    date_emb = Dense(5, activation='sigmoid', name='date_merged_emb')(date_emb)

    cat_layer = keras.layers.concatenate([holiday_flg, min_visitors, mean_visitors,
                    median_visitors, max_visitors, count_observations, rs1_x, rv1_x,
                    rs2_x, rv2_x, rs1_y, rv1_y, rs2_y, rv2_y,
                    total_reserv_sum, total_reserv_mean, total_reserv_dt_diff_mean,
                    date_int, var_max_lat, var_max_long, lon_plus_lat,
                    date_emb, air_area_name_emb, air_genre_name_emb,
                    air_area_code_emb, air_genre_code_emb, air_store_id_emb])

    m = Dense(hidden1_neurons, name='hidden1',
             kernel_initializer=keras.initializers.RandomNormal(mean=0.0,
                            stddev=0.05, seed=None))(cat_layer)
    m = keras.layers.LeakyReLU(alpha=0.2)(m)
    m = keras.layers.BatchNormalization()(m)
    
    m1 = Dense(hidden2_neurons, name='sub1')(m)
    m1 = keras.layers.LeakyReLU(alpha=0.2)(m1)
    m = Dense(1, activation='relu')(m1)

    inp_ten = [
        holiday_flg, min_visitors, mean_visitors, median_visitors, max_visitors, count_observations,
        rs1_x, rv1_x, rs2_x, rv2_x, rs1_y, rv1_y, rs2_y, rv2_y, total_reserv_sum, total_reserv_mean,
        total_reserv_dt_diff_mean, date_int, var_max_lat, var_max_long, lon_plus_lat,
        dow, year, month, air_store_id, air_area_code, air_genre_code
    ]
    inp_ten += air_area_name
    inp_ten += air_genre_name
    model = keras.Model(inp_ten, m)
    model.compile(loss='mse', optimizer='rmsprop', metrics=['acc'])

    return model

In [239]:
from keras.models import Sequential, Model

In [240]:
model4 = get_nn_complete_model(train, hidden2_neurons=12)

In [241]:
len(nn_train[0])

39

In [242]:
for i in range(5):
    model4.fit(nn_train[0], nn_train[1], epochs=3, verbose=1,
        batch_size=256, shuffle=True, validation_split=0.15)
    model4.fit(nn_train[0], nn_train[1], epochs=8, verbose=0,
        batch_size=256, shuffle=True)
print("Model4 trained")

Train on 5729 samples, validate on 1011 samples
Epoch 1/3
5729/5729 [==============================] - 2s 310us/step - loss: 5.4341 - acc: 0.0000e+00 - val_loss: 5.9333 - val_acc: 0.0000e+00
Epoch 2/3
5729/5729 [==============================] - 0s 34us/step - loss: 2.0983 - acc: 0.0000e+00 - val_loss: 2.2921 - val_acc: 0.0000e+00
Epoch 3/3
5729/5729 [==============================] - 0s 34us/step - loss: 1.4466 - acc: 0.0000e+00 - val_loss: 1.6116 - val_acc: 0.0000e+00
Train on 5729 samples, validate on 1011 samples
Epoch 1/3
5729/5729 [==============================] - 0s 36us/step - loss: 0.2055 - acc: 0.0000e+00 - val_loss: 0.2722 - val_acc: 0.0000e+00
Epoch 2/3
5729/5729 [==============================] - 0s 37us/step - loss: 0.2000 - acc: 0.0000e+00 - val_loss: 0.3845 - val_acc: 0.0000e+00
Epoch 3/3
5729/5729 [==============================] - 0s 34us/step - loss: 0.1965 - acc: 0.0000e+00 - val_loss: 0.3925 - val_acc: 0.0000e+00
Train on 5729 samples, validate on 1011 samples
Epo

In [243]:
preds5 = pd.Series(model4.predict(nn_train[0]).reshape(-1)).clip(0, 6.8).values

In [244]:
print('RMSE NeuralNetwork: ', RMSLE(np.log1p(train['visitors'].values), preds5))

RMSE NeuralNetwork:  0.4442205034375069


In [245]:
preds5 = pd.Series(model4.predict(nn_test[0]).reshape(-1)).clip(0, 6.8).values

InvalidArgumentError: indices[14,0] = 789 is not in [0, 789)
	 [[Node: air_store_id_emb/Gather = Gather[Tindices=DT_INT32, Tparams=DT_FLOAT, validate_indices=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](air_store_id_emb/embeddings/read, _arg_air_store_id_0_14)]]

Caused by op 'air_store_id_emb/Gather', defined at:
  File "/Users/suzukishinji/anaconda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/suzukishinji/anaconda/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/suzukishinji/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/suzukishinji/anaconda/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/suzukishinji/anaconda/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  Fi

In [248]:
#以下、各モデルをCVで4回
X = train[col]
y = pd.DataFrame()
y['visitors'] = np.log1p(train['visitors'].values)
testes =test[col]

In [249]:
X.sort_index(axis=1, inplace=True)
y.sort_index(axis=1,inplace=True)
testes.sort_index(axis=1,inplace=True)

/Users/suzukishinji/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/suzukishinji/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [250]:
X, X_hideout, y, y_hideout = model_selection.train_test_split(X, y, test_size = 0.13, random_state = 42)

In [251]:
K = 4
kf = model_selection.KFold(n_splits = K, random_state = 1, shuffle = True)
np.random.seed(1)

In [252]:
params = {'learning_rate': 0.1, 
        'n_estimators': 375,
         'max_depth' : 6,
         'min_samples_leaf': 2}
model1 = ensemble.GradientBoostingRegressor(**params)

In [253]:
y_test_pred1 = 0

In [254]:
print("Started CV at ", dt.datetime.now())
for i, (train_index, test_index) in enumerate(kf.split(X)):
    # Create data for this fold
    y_train, y_valid = y.iloc[train_index].copy(), y.iloc[test_index]
    X_train, X_valid = X.iloc[train_index, :].copy(), X.iloc[test_index, :].copy()
    X_test = testes
    print("\nFold ", i)

    fit_model = model1.fit(X_train, y_train)
    pred = model1.predict(X_valid)
    print('RMSLE GBM Regressor, validation set, fold ', i, ': ', RMSLE(y_valid, pred))

    pred_hideout = model1.predict(X_hideout)
    print('RMSLE GBM Regressor, hideout set, fold ', i, ': ', RMSLE(y_hideout, pred_hideout))
    print('Prediction length on validation set, GBM Regressor, fold ', i, ': ', len(pred))
    # Accumulate test set predictions

    pred = model1.predict(X_test)
    print('Prediction length on test set, GBM Regressor, fold ', i, ': ', len(pred))
    y_test_pred1 += pred

    del X_test, X_train, X_valid, y_train

print("Finished CV at ", dt.datetime.now())

y_test_pred1 /= K  # Average test set predictions
print("Finished average test set predictions at ", dt.datetime.now())

Started CV at  2018-02-06 14:52:57.381989

Fold  0


/Users/suzukishinji/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


RMSLE GBM Regressor, validation set, fold  0 :  0.46426761028114893
RMSLE GBM Regressor, hideout set, fold  0 :  0.4561170460023926
Prediction length on validation set, GBM Regressor, fold  0 :  1466
Prediction length on test set, GBM Regressor, fold  0 :  18062

Fold  1
RMSLE GBM Regressor, validation set, fold  1 :  0.4393993123459226
RMSLE GBM Regressor, hideout set, fold  1 :  0.4630895409882873
Prediction length on validation set, GBM Regressor, fold  1 :  1466
Prediction length on test set, GBM Regressor, fold  1 :  18062

Fold  2
RMSLE GBM Regressor, validation set, fold  2 :  0.44802343251179744
RMSLE GBM Regressor, hideout set, fold  2 :  0.45532103180991157
Prediction length on validation set, GBM Regressor, fold  2 :  1466
Prediction length on test set, GBM Regressor, fold  2 :  18062

Fold  3
RMSLE GBM Regressor, validation set, fold  3 :  0.46085482548447454
RMSLE GBM Regressor, hideout set, fold  3 :  0.47172054406435243
Prediction length on validation set, GBM Regressor,

In [255]:
params = {'n_jobs': -1,
        'n_neighbors': 4}

model2 = neighbors.KNeighborsRegressor(**params)

In [256]:
y_test_pred2 = 0

In [257]:
print("Started CV at ", dt.datetime.now())
for i, (train_index, test_index) in enumerate(kf.split(X)):
    # Create data for this fold
    y_train, y_valid = y.iloc[train_index].copy(), y.iloc[test_index]
    X_train, X_valid = X.iloc[train_index, :].copy(), X.iloc[test_index, :].copy()
    X_test = testes
    print("\nFold ", i)

    fit_model = model2.fit(X_train, y_train)
    pred = model2.predict(X_valid)
    print('RMSLE GBM Regressor, validation set, fold ', i, ': ', RMSLE(y_valid, pred))

    pred_hideout = model2.predict(X_hideout)
    print('RMSLE GBM Regressor, hideout set, fold ', i, ': ', RMSLE(y_hideout, pred_hideout))
    print('Prediction length on validation set, GBM Regressor, fold ', i, ': ', len(pred))
    # Accumulate test set predictions

    pred = model2.predict(X_test)
    print('Prediction length on test set, GBM Regressor, fold ', i, ': ', len(pred))
    y_test_pred2 += pred

    del X_test, X_train, X_valid, y_train

print("Finished CV at ", dt.datetime.now())

y_test_pred2 /= K  # Average test set predictions
print("Finished average test set predictions at ", dt.datetime.now())

Started CV at  2018-02-06 14:53:32.612382

Fold  0
RMSLE GBM Regressor, validation set, fold  0 :  0.5911623261448579
RMSLE GBM Regressor, hideout set, fold  0 :  0.5873523668889227
Prediction length on validation set, GBM Regressor, fold  0 :  1466
Prediction length on test set, GBM Regressor, fold  0 :  18062

Fold  1
RMSLE GBM Regressor, validation set, fold  1 :  0.5632223617043908
RMSLE GBM Regressor, hideout set, fold  1 :  0.5872864380052101
Prediction length on validation set, GBM Regressor, fold  1 :  1466
Prediction length on test set, GBM Regressor, fold  1 :  18062

Fold  2
RMSLE GBM Regressor, validation set, fold  2 :  0.5760207511509177
RMSLE GBM Regressor, hideout set, fold  2 :  0.5832090942912816
Prediction length on validation set, GBM Regressor, fold  2 :  1466
Prediction length on test set, GBM Regressor, fold  2 :  18062

Fold  3
RMSLE GBM Regressor, validation set, fold  3 :  0.5601378679997298
RMSLE GBM Regressor, hideout set, fold  3 :  0.5853159037128044
Predi

In [258]:
params = {'n_estimators': 200, 
        'max_depth': 10,
        'min_samples_split': 200,
        'min_samples_leaf': 50,
        'learning_rate': 0.2,
        'max_features':  'sqrt',
        'subsample': 0.8,
        'loss': 'ls'}

model3 = XGBRegressor(**params)

In [259]:
y_test_pred3 = 0

In [260]:
print("Started CV at ", dt.datetime.now())
for i, (train_index, test_index) in enumerate(kf.split(X)):
    # Create data for this fold
    y_train, y_valid = y.iloc[train_index].copy(), y.iloc[test_index]
    X_train, X_valid = X.iloc[train_index, :].copy(), X.iloc[test_index, :].copy()
    X_test = testes
    print("\nFold ", i)

    fit_model = model3.fit(X_train, y_train)
    pred = model3.predict(X_valid)
    print('RMSLE GBM Regressor, validation set, fold ', i, ': ', RMSLE(y_valid, pred))

    pred_hideout = model3.predict(X_hideout)
    print('RMSLE GBM Regressor, hideout set, fold ', i, ': ', RMSLE(y_hideout, pred_hideout))
    print('Prediction length on validation set, GBM Regressor, fold ', i, ': ', len(pred))
    # Accumulate test set predictions

    pred = model3.predict(X_test)
    print('Prediction length on test set, GBM Regressor, fold ', i, ': ', len(pred))
    y_test_pred3 += pred

    del X_test, X_train, X_valid, y_train

print("Finished CV at ", dt.datetime.now())

y_test_pred3 /= K  # Average test set predictions
print("Finished average test set predictions at ", dt.datetime.now())

Started CV at  2018-02-06 14:53:35.178109

Fold  0
RMSLE GBM Regressor, validation set, fold  0 :  0.4820664774298244
RMSLE GBM Regressor, hideout set, fold  0 :  0.46676638040007257
Prediction length on validation set, GBM Regressor, fold  0 :  1466
Prediction length on test set, GBM Regressor, fold  0 :  18062

Fold  1
RMSLE GBM Regressor, validation set, fold  1 :  0.4762476114370208
RMSLE GBM Regressor, hideout set, fold  1 :  0.4849414211919609
Prediction length on validation set, GBM Regressor, fold  1 :  1466
Prediction length on test set, GBM Regressor, fold  1 :  18062

Fold  2
RMSLE GBM Regressor, validation set, fold  2 :  0.4727376461376971
RMSLE GBM Regressor, hideout set, fold  2 :  0.47255336167752104
Prediction length on validation set, GBM Regressor, fold  2 :  1466
Prediction length on test set, GBM Regressor, fold  2 :  18062

Fold  3
RMSLE GBM Regressor, validation set, fold  3 :  0.48835520363576895
RMSLE GBM Regressor, hideout set, fold  3 :  0.5003288638662243
Pr

In [261]:
#LGBMだけ入れるデータを少し加工
X = train[col]
y = pd.DataFrame()
y['visitors'] = np.log1p(train['visitors'].values)
testes =test[col]
X.sort_index(axis=1, inplace=True)
y.sort_index(axis=1,inplace=True)
testes.sort_index(axis=1,inplace=True)
X, X_hideout, y, y_hideout = model_selection.train_test_split(X, y, test_size = 0.13, random_state = 42)
y = y['visitors']
y_hideout = y_hideout['visitors']

/Users/suzukishinji/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/suzukishinji/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [262]:
import lightgbm as lgb
params = {'learning_rate': 0.05, 
        'n_estimators': 900,
         'max_depth' : 8,
         'boosting_type': 'gbdt',
         'num_leaves' : 80,
         'max_bin' : 255,
         'seed' : 42,
         'nthread' : -1,
         'silent' : True}

model4 = lgb.LGBMRegressor(**params)

In [263]:
y_test_pred4 = 0

In [264]:
print("Started CV at ", dt.datetime.now())
for i, (train_index, test_index) in enumerate(kf.split(X)):
    # Create data for this fold
    y_train, y_valid = y.iloc[train_index].copy(), y.iloc[test_index]
    X_train, X_valid = X.iloc[train_index, :].copy(), X.iloc[test_index, :].copy()
    X_test = testes
    print("\nFold ", i)

    fit_model = model4.fit(X_train, y_train)
    pred = model4.predict(X_valid)
    print('RMSLE GBM Regressor, validation set, fold ', i, ': ', RMSLE(y_valid, pred))

    pred_hideout = model4.predict(X_hideout)
    print('RMSLE GBM Regressor, hideout set, fold ', i, ': ', RMSLE(y_hideout, pred_hideout))
    print('Prediction length on validation set, GBM Regressor, fold ', i, ': ', len(pred))
    # Accumulate test set predictions

    pred = model4.predict(X_test)
    print('Prediction length on test set, GBM Regressor, fold ', i, ': ', len(pred))
    y_test_pred4 += pred

    del X_test, X_train, X_valid, y_train

print("Finished CV at ", dt.datetime.now())

y_test_pred4 /= K  # Average test set predictions
print("Finished average test set predictions at ", dt.datetime.now())

/Users/suzukishinji/anaconda/lib/python3.6/site-packages/lightgbm/sklearn.py:278: LGBMDeprecationWarning: The `seed` parameter is deprecated and will be removed in 2.0.12 version. Please use `random_state` instead.
  'Please use `random_state` instead.', LGBMDeprecationWarning)
/Users/suzukishinji/anaconda/lib/python3.6/site-packages/lightgbm/sklearn.py:281: LGBMDeprecationWarning: The `nthread` parameter is deprecated and will be removed in 2.0.12 version. Please use `n_jobs` instead.
  'Please use `n_jobs` instead.', LGBMDeprecationWarning)
/Users/suzukishinji/anaconda/lib/python3.6/site-packages/lightgbm/basic.py:642: UserWarning: max_bin keyword has been found in `params` and will be ignored. Please use max_bin argument of the Dataset constructor to pass this parameter.
  'Please use {0} argument of the Dataset constructor to pass this parameter.'.format(key))
/Users/suzukishinji/anaconda/lib/python3.6/site-packages/lightgbm/basic.py:648: LGBMDeprecationWarning: The `max_bin` param

Started CV at  2018-02-06 14:54:06.812276

Fold  0
RMSLE GBM Regressor, validation set, fold  0 :  0.45361773127407196
RMSLE GBM Regressor, hideout set, fold  0 :  0.4468174985613663
Prediction length on validation set, GBM Regressor, fold  0 :  1466
Prediction length on test set, GBM Regressor, fold  0 :  18062

Fold  1


/Users/suzukishinji/anaconda/lib/python3.6/site-packages/lightgbm/sklearn.py:278: LGBMDeprecationWarning: The `seed` parameter is deprecated and will be removed in 2.0.12 version. Please use `random_state` instead.
  'Please use `random_state` instead.', LGBMDeprecationWarning)
/Users/suzukishinji/anaconda/lib/python3.6/site-packages/lightgbm/sklearn.py:281: LGBMDeprecationWarning: The `nthread` parameter is deprecated and will be removed in 2.0.12 version. Please use `n_jobs` instead.
  'Please use `n_jobs` instead.', LGBMDeprecationWarning)
/Users/suzukishinji/anaconda/lib/python3.6/site-packages/lightgbm/basic.py:642: UserWarning: max_bin keyword has been found in `params` and will be ignored. Please use max_bin argument of the Dataset constructor to pass this parameter.
  'Please use {0} argument of the Dataset constructor to pass this parameter.'.format(key))
/Users/suzukishinji/anaconda/lib/python3.6/site-packages/lightgbm/basic.py:648: LGBMDeprecationWarning: The `max_bin` param

RMSLE GBM Regressor, validation set, fold  1 :  0.4388016153477257
RMSLE GBM Regressor, hideout set, fold  1 :  0.45805777988922913
Prediction length on validation set, GBM Regressor, fold  1 :  1466
Prediction length on test set, GBM Regressor, fold  1 :  18062

Fold  2


/Users/suzukishinji/anaconda/lib/python3.6/site-packages/lightgbm/sklearn.py:278: LGBMDeprecationWarning: The `seed` parameter is deprecated and will be removed in 2.0.12 version. Please use `random_state` instead.
  'Please use `random_state` instead.', LGBMDeprecationWarning)
/Users/suzukishinji/anaconda/lib/python3.6/site-packages/lightgbm/sklearn.py:281: LGBMDeprecationWarning: The `nthread` parameter is deprecated and will be removed in 2.0.12 version. Please use `n_jobs` instead.
  'Please use `n_jobs` instead.', LGBMDeprecationWarning)
/Users/suzukishinji/anaconda/lib/python3.6/site-packages/lightgbm/basic.py:642: UserWarning: max_bin keyword has been found in `params` and will be ignored. Please use max_bin argument of the Dataset constructor to pass this parameter.
  'Please use {0} argument of the Dataset constructor to pass this parameter.'.format(key))
/Users/suzukishinji/anaconda/lib/python3.6/site-packages/lightgbm/basic.py:648: LGBMDeprecationWarning: The `max_bin` param

RMSLE GBM Regressor, validation set, fold  2 :  0.4446085213118169
RMSLE GBM Regressor, hideout set, fold  2 :  0.45312883017780575
Prediction length on validation set, GBM Regressor, fold  2 :  1466
Prediction length on test set, GBM Regressor, fold  2 :  18062

Fold  3


/Users/suzukishinji/anaconda/lib/python3.6/site-packages/lightgbm/sklearn.py:278: LGBMDeprecationWarning: The `seed` parameter is deprecated and will be removed in 2.0.12 version. Please use `random_state` instead.
  'Please use `random_state` instead.', LGBMDeprecationWarning)
/Users/suzukishinji/anaconda/lib/python3.6/site-packages/lightgbm/sklearn.py:281: LGBMDeprecationWarning: The `nthread` parameter is deprecated and will be removed in 2.0.12 version. Please use `n_jobs` instead.
  'Please use `n_jobs` instead.', LGBMDeprecationWarning)
/Users/suzukishinji/anaconda/lib/python3.6/site-packages/lightgbm/basic.py:642: UserWarning: max_bin keyword has been found in `params` and will be ignored. Please use max_bin argument of the Dataset constructor to pass this parameter.
  'Please use {0} argument of the Dataset constructor to pass this parameter.'.format(key))
/Users/suzukishinji/anaconda/lib/python3.6/site-packages/lightgbm/basic.py:648: LGBMDeprecationWarning: The `max_bin` param

RMSLE GBM Regressor, validation set, fold  3 :  0.44880941054143975
RMSLE GBM Regressor, hideout set, fold  3 :  0.457049025099983
Prediction length on validation set, GBM Regressor, fold  3 :  1465
Prediction length on test set, GBM Regressor, fold  3 :  18062
Finished CV at  2018-02-06 14:54:34.531476
Finished average test set predictions at  2018-02-06 14:54:34.532753


In [265]:
#モデル２のpredだけ２次元で出てくるので加工
dataset = np.zeros(shape=(len(y_test_pred2), 1))
feature_names = ['visitors']
tmp = pd.DataFrame(dataset,columns=feature_names)
tmp['visitors'] = y_test_pred2

In [266]:
pre4 = model4.predict(X)

In [269]:
print(pred_513.shape)
print(pred_316.shape)
print(pred_gw.shape)

(19812,)
(12207,)
(18062,)


In [88]:
#pred_513 =  y_test_pred1*0.3 + tmp['visitors'] *0.05 + y_test_pred3*0.1+ y_test_pred4*0.35+preds5*0.2

In [177]:
#pred_316 =  y_test_pred1*0.3 + tmp['visitors'] *0.05 + y_test_pred3*0.1+ y_test_pred4*0.35+preds5*0.2

In [267]:
pred_gw =  y_test_pred1*0.3 + tmp['visitors'] *0.15 + y_test_pred3*0.2+ y_test_pred4*0.35#+preds5*0.2

In [268]:
pred_gw.shape

(18062,)

In [270]:
store_gw = pd.read_csv('/Users/suzukishinji/kaggle/recluit/sample_submission-3.csv')
store_gw['visitors'] =pred_gw
store_gw = store_gw.fillna(0)
store_316 = pd.read_csv('/Users/suzukishinji/kaggle/recluit/tes_store_316.csv')
store_316['visitors'] =pred_316 
store_513 = pd.read_csv('/Users/suzukishinji/kaggle/recluit/tes_store_513.csv')
store_513['visitors'] =pred_513

In [271]:
store_gw.tail()

,id,visitors
32014,air_fff68b929994bfbd_2017-05-27,0.0
32015,air_fff68b929994bfbd_2017-05-28,0.0
32016,air_fff68b929994bfbd_2017-05-29,0.0
32017,air_fff68b929994bfbd_2017-05-30,0.0
32018,air_fff68b929994bfbd_2017-05-31,0.0


In [280]:
temp_gw = store_gw

In [283]:
temp_gw.rename(columns ={'visitors':'gw_visitors'})
temp_gw.head()

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,1.312121
1,air_00a91d42b08b08d9_2017-04-24,1.321203
2,air_00a91d42b08b08d9_2017-04-25,1.318761
3,air_00a91d42b08b08d9_2017-04-26,1.314564
4,air_00a91d42b08b08d9_2017-04-27,1.302656


In [272]:
all_store = pd.concat([store_316,store_513])

In [273]:
sample = pd.read_csv('/Users/suzukishinji/kaggle/recluit/sample_submission-3.csv')
test_air_store_id = sample['id']
sub_stores = pd.DataFrame()
sub_stores["id"] = test_air_store_id
sub_stores["visitors"] = test['visitors']

In [285]:
rename_all = all_store.rename(columns ={'visitors':'all_visitors'})
rename_all.head()

,id,all_visitors
0,air_0241aa3964b7f861_2017-04-23,2.210916
1,air_0241aa3964b7f861_2017-04-24,2.150173
2,air_0241aa3964b7f861_2017-04-25,2.138857
3,air_0241aa3964b7f861_2017-04-26,2.209877
4,air_0241aa3964b7f861_2017-04-27,1.307983


In [286]:
all_all = pd.merge(store_gw, rename_all,on=('id'), how='left')
all_all.head()

,id,visitors,all_visitors
0,air_00a91d42b08b08d9_2017-04-23,1.312121,1.400415
1,air_00a91d42b08b08d9_2017-04-24,1.321203,3.106879
2,air_00a91d42b08b08d9_2017-04-25,1.318761,3.299502
3,air_00a91d42b08b08d9_2017-04-26,1.314564,3.355776
4,air_00a91d42b08b08d9_2017-04-27,1.302656,3.446238


In [288]:
all_all['sub_visitors'] = all_all['visitors']*0.2 + all_all['all_visitors']*0.8
all_all.head()

,id,visitors,all_visitors,sub_visitors
0,air_00a91d42b08b08d9_2017-04-23,1.382756,1.400415,1.396883
1,air_00a91d42b08b08d9_2017-04-24,2.749744,3.106879,3.035452
2,air_00a91d42b08b08d9_2017-04-25,2.903354,3.299502,3.220273
3,air_00a91d42b08b08d9_2017-04-26,2.947533,3.355776,3.274127
4,air_00a91d42b08b08d9_2017-04-27,3.017521,3.446238,3.360494


In [289]:
temp = all_all.drop(['visitors','all_visitors'], axis = 1)
all_store = temp.rename(columns ={'sub_visitors':'visitors'})
all_store.head()

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,1.396883
1,air_00a91d42b08b08d9_2017-04-24,3.035452
2,air_00a91d42b08b08d9_2017-04-25,3.220273
3,air_00a91d42b08b08d9_2017-04-26,3.274127
4,air_00a91d42b08b08d9_2017-04-27,3.360494


In [290]:
test =pd.read_csv('/Users/suzukishinji/kaggle/recluit/sample_submission-3.csv')
test = test.drop('visitors', axis = 1)
test = pd.merge(test, all_store,on=('id'), how='left')
test = test.fillna(0)
test['visitors'] = np.expm1(test['visitors']).clip(lower =0.)
test.head()

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,3.042581
1,air_00a91d42b08b08d9_2017-04-24,19.810383
2,air_00a91d42b08b08d9_2017-04-25,24.034946
3,air_00a91d42b08b08d9_2017-04-26,25.420152
4,air_00a91d42b08b08d9_2017-04-27,27.803428


In [482]:
##割合変更（ここはあまり試していません）
test['visitors'] =  y_test_pred1*0.3 + tmp['visitors'] *0.05 + y_test_pred3*0.1+ y_test_pred4*0.35+preds5*0.2
#test['visitors'] = y_test_pred1*0.15+ tmp['visitors'] *0.1 + y_test_pred3*0.1+ y_test_pred4*0.4+preds5*0.25
test['visitors'].mean()
#test['visitors'] = np.expm1(test['visitors']).clip(lower =0.)

2.157909899573929

In [366]:
type(y_test_pred1)

numpy.ndarray

In [379]:
#単品のスコアを見るため
test['visitors'] =  y_test_pred3
test['visitors'] = np.expm1(test['visitors']).clip(lower =0.)
sample = pd.read_csv('/Users/suzukishinji/kaggle/recluit/sample_submission-3.csv')
test_air_store_id = sample['id']
df_result = pd.DataFrame()
df_result["id"] = test_air_store_id
df_result["visitors"] = test['visitors']
df_result.to_csv("submit_0126_model3.csv",index = False)
submit_1 = pd.read_csv('/Users/suzukishinji/kaggle/recluit/submit_0126_model3.csv')
submit_1.head()

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,3.058
1,air_00a91d42b08b08d9_2017-04-24,19.907
2,air_00a91d42b08b08d9_2017-04-25,23.276
3,air_00a91d42b08b08d9_2017-04-26,27.628
4,air_00a91d42b08b08d9_2017-04-27,30.793


In [291]:
#ここからの加重平均は特にさわっていません
sub1 = test[['id','visitors']].copy()

In [292]:
date_info = pd.read_csv('/Users/suzukishinji/kaggle/recluit/date_info.csv')
air_visit_data = pd.read_csv('/Users/suzukishinji/kaggle/recluit/air_visit_data.csv')
sample_submission = pd.read_csv('/Users/suzukishinji/kaggle/recluit/sample_submission-3.csv')

In [293]:
dfs = { re.search('/([^/\.]*)\.csv', fn).group(1):
       pd.read_csv(fn) for fn in glob.glob('/Users/suzukishinji/kaggle/recluit/*.csv')}

In [294]:
for k, v in dfs.items(): locals()[k] = v

In [295]:
wkend_holidays = date_info.apply((lambda x: (x.day_of_week == 'Sunday' or x.day_of_week == 'Saturday') and x.holiday_flg ==1), axis = 1)

In [296]:
date_info.loc[wkend_holidays, 'holiday_flg'] = 0
date_info['weight'] = ((date_info.index + 1) / len(date_info))**5
visit_data = air_visit_data.merge(date_info, left_on= 'visit_date', right_on = 'calendar_date', how = 'left')
visit_data.drop('calendar_date', axis = 1, inplace = True)
visit_data['visitors'] = visit_data.visitors.map(pd.np.log1p)
wmean = lambda x: ((x.weight * x.visitors).sum()/x.weight.sum())
visitors = visit_data.groupby(['air_store_id', 'day_of_week', 'holiday_flg']).apply(wmean).reset_index()
visitors.rename(columns = {0:'visitors'}, inplace = True)

In [297]:
sample_submission['air_store_id'] = sample_submission.id.map(lambda x : '_'.join(x.split('_')[:-1]))
sample_submission['calendar_date'] = sample_submission.id.map(lambda x: x.split('_')[2])
sample_submission.drop('visitors', axis=1, inplace=True)
sample_submission = sample_submission.merge(date_info, on='calendar_date', how='left')
sample_submission = sample_submission.merge(visitors, on=[
    'air_store_id', 'day_of_week', 'holiday_flg'], how='left')

In [298]:
missings = sample_submission.visitors.isnull()
sample_submission.loc[missings, 'visitors'] = sample_submission[missings].merge(visitors[visitors.holiday_flg == 0], on = ('air_store_id', 'day_of_week'),how = 'left')['visitors_y'].values
missings = sample_submission.visitors.isnull()
sample_submission.loc[missings, 'visitors'] = sample_submission[missings].merge(visitors[['air_store_id', 'visitors']].groupby('air_store_id').mean().reset_index(), on = 'air_store_id',how = 'left')['visitors_y'].values

In [299]:
sample_submission['visitors'] = sample_submission.visitors.map(pd.np.expm1)
sub2 = sample_submission[['id', 'visitors']].copy()
sub_merge = pd.merge(sub1, sub2, on = 'id', how = 'inner')

In [300]:
sub_merge['visitors'] = 0.6*sub_merge['visitors_x'] + 0.4*sub_merge['visitors_y']*1.1
sub_merge[['id', 'visitors']].to_csv('submission_0206_1.csv', index = False)

In [301]:
#ちゃんと出来ているか確認
submit_1 = pd.read_csv('/Users/suzukishinji/kaggle/recluit/submission_0206_1.csv')
submit_1.head()

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,2.705548
1,air_00a91d42b08b08d9_2017-04-24,22.279748
2,air_00a91d42b08b08d9_2017-04-25,26.223145
3,air_00a91d42b08b08d9_2017-04-26,27.396496
4,air_00a91d42b08b08d9_2017-04-27,30.453901


In [1267]:
submit_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32019 entries, 0 to 32018
Data columns (total 2 columns):
id          32019 non-null object
visitors    32019 non-null float64
dtypes: float64(1), object(1)
memory usage: 500.4+ KB
